<a href="https://colab.research.google.com/github/willarakaki/dio-tradutor-azure-desafio/blob/main/azure_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando um tradutor de artigos com OpenAI Azure


In [1]:
!pip install requests beautifulsoup4 openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.1/500.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9


In [8]:
import requests
from bs4 import BeautifulSoup

def extrair_texto_do_artigo(url):
    response = requests.get(url)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
      texto = soup.get_text(separator= ' ')
      #Limpar texto
      linhas = (line.strip() for line in texto.splitlines())
      parts = (phrase.strip() for line in linhas for phrase in line.split("  "))
      texto_limpo = '\n'.join(line for line in linhas if line)
      return texto_limpo
    else:
      print(f"Failed to fetch the URL. Status code: {response.status_code}")
      return None

    text = soup.get_text()
    return text

extrair_texto_do_artigo('https://dev.to/javz/why-learning-basic-robotics-made-me-a-better-software-engineer-in-the-age-of-ai-fdh')

'Why Learning Basic Robotics Made Me a Better Software Engineer in the Age of AI - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by Algolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nBoost\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nJulien Avezou\nPosted on  Feb 10\nWhy Learning Basic Robotics Made Me a Better Software Engineer in the Age of AI\n# discuss\n# ai\n# robotics\n# programming\nI recently bought a starter kit for Arduino to learn robotics. I was curious to learn some basic concepts and get inspired by what\'s possible to build, not just in software but also in the physical world. I want to emphasize that my current goal is not to pursue robotics at a deep level, it\'s simply to expose myself to the basic concepts so I can develop my intuitions for the field.\nAs I wr

In [11]:
from langchain_openai.chat_models import AzureChatOpenAI
from google.colab import userdata

client = AzureChatOpenAI(
    azure_endpoint= "https://openai-dio-bootcamp-eastus-001.openai.azure.com/",
    api_key= userdata.get("OPENAI_AZURE_KEY"),
    api_version= "2024-12-01-preview",
    deployment_name= "o4-mini",
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o {text} para o idioma {lang} e responda em MARKDOWN")
  ]

  response = client.invoke(messages)
  print(response.content)
  return response.content

translate_article("Let's see if the deployment was succeeded.", "portugues")

```markdown
Vamos ver se a implantação foi bem-sucedida.
```


'```markdown\nVamos ver se a implantação foi bem-sucedida.\n```'

In [12]:
url = "https://dev.to/javz/why-learning-basic-robotics-made-me-a-better-software-engineer-in-the-age-of-ai-fdh"
text = extrair_texto_do_artigo(url)
article = translate_article(text, "pt-br")

print(article)

```markdown
# Por que aprender robótica básica me tornou um engenheiro de software melhor na era da IA

*Publicado em 10 de fev.*

Tags: `discuss` `ai` `robotics` `programming`

---

Recentemente comprei um kit iniciante de Arduino para aprender robótica. Eu estava curioso para absorver alguns conceitos básicos e me inspirar no que é possível construir, não só em software, mas também no mundo físico. Quero enfatizar que meu objetivo atual não é me aprofundar em robótica a nível avançado, mas apenas me expor aos conceitos básicos para desenvolver minhas intuições na área.

Ao concluir o curso, gostaria de compartilhar minhas impressões sobre como aprender robótica tem agregado valor à minha trajetória como desenvolvedor, especialmente nesta era de IA.

Antes de falar sobre minhas conclusões, um pouco de contexto para quem tiver interesse no conteúdo do kit. O curso incluído vinha com uma série de projetos em Arduino que se complementam gradualmente, com explicações claras, esquemas e ex